In [1]:
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
from PIL import ImageDraw, ImageFont
from skimage.measure import label, regionprops
import cv2
import os
from typing import Tuple
import random


def find_field_center(mask: np.ndarray, num_samples: int = 50) -> Tuple[int, int]:
    ys, xs = np.where(mask)
    if len(xs) == 0:
        raise ValueError("The provided mask is empty.")

    min_x, max_x = xs.min(), xs.max()
    min_y, max_y = ys.min(), ys.max()

    best_point: Tuple[int, int] = (min_x, min_y)
    best_score: float = -1.0

    for _ in range(num_samples): # Es werden num_samples Punkte zufällig ausgewählt. Standardmässig habe ich dies auf 50 gesetzt.
        while True:
            x = random.randint(min_x, max_x)
            y = random.randint(min_y, max_y)
            if mask[y, x]:
                break

        # North
        dn = 0
        for yy in range(y, min_y - 1, -1):
            if not mask[yy, x]:
                dn = y - yy
                break

        # South
        ds = 0
        for yy in range(y, max_y + 1):
            if not mask[yy, x]:
                ds = yy - y
                break

        # West
        dw = 0
        for xx in range(x, min_x - 1, -1):
            if not mask[y, xx]:
                dw = x - xx
                break

        # East
        de = 0
        for xx in range(x, max_x + 1):
            if not mask[y, xx]:
                de = xx - x
                break

        # Durschnittlicher Abstand
        score = (dn + ds + dw + de) / 4.0
        if score > best_score:
            best_score = score
            best_point = (x, y)

    return best_point


def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return np.array(image)

def quantize_colors(image_array, n_colors):
    # Apply Gaussian blur to smooth transitions and reduce small color noise
    blurred = cv2.GaussianBlur(image_array, (5, 5), 0)

    h, w, c = blurred.shape
    image_flat = blurred.reshape(-1, 3)
    kmeans = KMeans(n_clusters=n_colors, random_state=42).fit(image_flat)
    labels = kmeans.labels_
    quantized_flat = kmeans.cluster_centers_[labels].astype('uint8')
    quantized_image = quantized_flat.reshape(h, w, 3)
    return quantized_image, labels.reshape(h, w), kmeans.cluster_centers_

def detect_regions_by_color(quantized_image):
    h, w, _ = quantized_image.shape
    labeled_image = np.zeros((h, w), dtype=int)
    current_label = 1

    unique_colors = np.unique(quantized_image.reshape(-1, 3), axis=0)
    for color in unique_colors:
        mask = np.all(quantized_image == color, axis=-1)
        labeled_mask = label(mask, connectivity=1)
        labeled_mask[labeled_mask > 0] += current_label - 1
        labeled_image += labeled_mask
        current_label = labeled_image.max() + 1

    return labeled_image

def filter_small_regions(labeled_image, min_size=100):
    new_labeled = np.zeros_like(labeled_image)
    current_label = 1
    for region in regionprops(labeled_image):
        if region.area >= min_size:
            coords = region.coords
            for y, x in coords:
                new_labeled[y, x] = current_label
            current_label += 1
    return new_labeled

def assign_numbers(cluster_centers):
    color_to_number = {tuple(center.astype(int)): idx+1 for idx, center in enumerate(cluster_centers)}
    return color_to_number

def save_legend(color_to_number, output_folder):
    legend_path = os.path.join(output_folder, "legend.txt")
    with open(legend_path, "w") as f:
        for color, number in color_to_number.items():
            f.write(f"{number}: RGB{color}\n")
    print(f"Legend saved to {legend_path}")


def overlay_numbers(image_array: np.ndarray,
                    labeled_regions: np.ndarray,
                    color_to_number: dict,
                    font_path: str = None,
                    font_size: int = 20,
                    num_samples: int = 200) -> Image.Image:
    image = Image.fromarray(image_array)
    draw  = ImageDraw.Draw(image)
    if font_path:
        font = ImageFont.truetype(font_path, font_size)
    else:
        font = ImageFont.load_default()

    # Pro Region einmal durchgehen
    for region in regionprops(labeled_regions):
        # War bereits vorher so im code. Ich nehme an, dass ist der Hintergrund?!
        if region.label == 0:
            continue

        mask_full = (labeled_regions == region.label)

        # Hier wird meine neue Funktion aufgerufen.
        cx, cy = find_field_center(mask_full, num_samples=num_samples)

        # Übernahme von Pascal seinem Code
        color = tuple(image_array[mask_full][0])
        number = color_to_number.get(color, '?')
        draw.text((cx, cy), str(number), fill='black', font=font)

    return image

def OLD_overlay_numbers(image_array, labeled_regions, color_to_number):
    image = Image.fromarray(image_array)
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()



    for region_label in np.unique(labeled_regions):
        if region_label == 0:
            continue
        mask = labeled_regions == region_label
        coords = np.argwhere(mask)
        y, x = coords.mean(axis=0).astype(int)
        color = tuple(image_array[mask][0])
        number = color_to_number.get(color, '?')
        draw.text((x, y), str(number), fill='black', font=font)

    return image

def create_paint_by_numbers(image_path, save_path, n_colors):
    output_folder = save_path
    os.makedirs(output_folder, exist_ok=True)

    image_array = load_image(image_path)
    quantized_image, label_image, cluster_centers = quantize_colors(image_array, n_colors)
    labeled_regions = detect_regions_by_color(quantized_image)
    labeled_regions = filter_small_regions(labeled_regions, min_size=150)  # tweak size
    color_to_number = assign_numbers(cluster_centers)
    final_image = overlay_numbers(quantized_image, labeled_regions, color_to_number)

    # Save image
    output_image_path = os.path.join(output_folder, "paint_by_numbers_output.png")
    final_image.save(output_image_path)
    print(f"Paint-by-numbers image saved to {output_image_path}")

    # Save legend
    save_legend(color_to_number, output_folder)



In [3]:
image_path = '../test-images-mnz/castle_and_guards.jpg'
save_path = '../target'
number_colors = 16

create_paint_by_numbers(image_path, save_path, number_colors)

Paint-by-numbers image saved to ../target/paint_by_numbers_output.png
Legend saved to ../target/legend.txt
